## Integrantes:

- Kevin Pareja (20196318)

- Elian Tongombol (20196453)

- Paola Aranda (20196052)

- María Alejandra Colán (20190515)

In [2]:
# Importamos las librerias necesarias
import pandas as pd 
import numpy as np
import re 
from tqdm import tqdm  # controlar el tiempo en un loop
import os

# Librerias correspondientes al modelo de regresión lineal 
import statsmodels.api as sm  # linear regression utiliza todas las columnas de base de datos 
import statsmodels.formula.api as smf  # linear regression usa uan formula
from sklearn import datasets, linear_model # models 
from sklearn.metrics import mean_squared_error, r2_score
from linearmodels.iv import IV2SLS # for IV regression

import warnings
warnings.filterwarnings('ignore') # eliminar warning messages 

# Incluimos la libreriia que nos servía para exportar las tablas a Latex
from pystout import pystout

In [13]:
# Cargando la base de datos correspondiente 
datos = pd.read_stata(r"../../data/trabajo_final/replicacion/SerraPorterAEJ.dta",
                           convert_categoricals=False)
datos

,anonid,took_fall,took_year,tookanother,numeconclass,econmajor,Major,Major_STEM,Major_Business,Major_Finance,...,ecojobs_b,ecojobs_c,ecojobs_d,ecojobs_e,ecojobs_f,ecojobs_g,ecojobs_h,major_current_a,major_current_b,survey_econmajor
0,4.0,0.0,0.0,0.0,0.0,0.0,Mathematics - BS,1.0,0.0,0.0,...,Entrepreneur,Journalist/ Correspondent,Working for marketplace on NPR,Business consultant,,,,Mathematics,,0.0
1,5.0,0.0,0.0,0.0,0.0,0.0,Biological Sciences - BS,1.0,0.0,0.0,...,Tax advisor,Managerial Position,,,,,,Biology,,0.0
2,7.0,1.0,1.0,1.0,13.0,1.0,Economics w/ Fina Appl - BS,0.0,0.0,0.0,...,,,,,,,,,,NaN
3,9.0,0.0,0.0,0.0,0.0,0.0,Journalism - BA,0.0,0.0,0.0,...,,,,,,,,Journalism,Communications,0.0
4,10.0,0.0,0.0,1.0,1.0,0.0,Markets & Culture - BA,0.0,0.0,0.0,...,Investment Banker,,,,,,,Markets and Culture,Economics,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
622,1386.0,0.0,0.0,0.0,0.0,0.0,Markets & Culture - BA,0.0,0.0,0.0,...,,,,,,,,,,NaN
623,1388.0,0.0,0.0,0.0,0.0,0.0,Economics w/ Fina Appl - PMJ,0.0,0.0,0.0,...,,,,,,,,Business,,0.0
624,1389.0,0.0,0.0,0.0,0.0,0.0,Accounting - BBA,0.0,1.0,0.0,...,Finance,Bank,Government,,,,,Economics,Public Relations,1.0
625,1393.0,0.0,1.0,1.0,10.0,1.0,Economics - BA,0.0,0.0,0.0,...,Professor,Economist,,,,,,Entrepreneurship,Finance Economics,0.0


In [12]:
# Incluimos el resumen de los estadísticos
datos.describe()

# Analizando el tipo de variables
datos.info() # A partir de esta información se escogerá la información solicitada

datos.dtypes


<class 'pandas.core.frame.DataFrame'>
Int64Index: 627 entries, 0 to 626
Data columns (total 49 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   anonid            627 non-null    float32
 1   took_fall         627 non-null    float32
 2   took_year         627 non-null    float32
 3   tookanother       627 non-null    float32
 4   numeconclass      627 non-null    float32
 5   econmajor         627 non-null    float32
 6   Major             627 non-null    object 
 7   Major_STEM        627 non-null    float32
 8   Major_Business    627 non-null    float32
 9   Major_Finance     627 non-null    float32
 10  Major_Marketing   627 non-null    float32
 11  Major_SocSc       627 non-null    float32
 12  Major_Arts        627 non-null    float32
 13  Major_Comm        627 non-null    float32
 14  Major_Hum         627 non-null    float32
 15  female            627 non-null    float32
 16  greek             501 non-null    float64
 1

anonid              float32
took_fall           float32
took_year           float32
tookanother         float32
numeconclass        float32
econmajor           float32
Major                object
Major_STEM          float32
Major_Business      float32
Major_Finance       float32
Major_Marketing     float32
Major_SocSc         float32
Major_Arts          float32
Major_Comm          float32
Major_Hum           float32
female              float32
greek               float64
econ_hs             float32
varsity             float64
class_fe            float32
class_fe2           float32
yr_2016             float32
treatment_class     float32
treat2016           float32
female_prof         float32
instate             float32
freshman            float32
american            float32
ACumGPA             float64
gradePrinciples     float32
small_class         float32
grade3301           float32
fem2016             float32
femtreat16          float32
female_treat        float32
Testscore3301       

###  1. Estadisticas descriptivas

In [15]:
# Seleccionamos las variables para las estadísticas descriptivas
table1 = data.loc[:,["american", "instate", "freshman","ACumGPA", "greek", "econ_hs", "varsity"]]
table1 

,american,instate,freshman,ACumGPA,greek,econ_hs,varsity
0,1.0,1.0,0.0,3.267,0.0,1.0,0.0
1,1.0,1.0,0.0,3.358,0.0,1.0,0.0
2,1.0,0.0,1.0,1.775,NaN,NaN,NaN
3,1.0,1.0,0.0,3.654,1.0,1.0,0.0
4,1.0,1.0,0.0,3.000,1.0,1.0,0.0
...,...,...,...,...,...,...,...
622,1.0,0.0,1.0,2.653,NaN,NaN,NaN
623,0.0,0.0,1.0,3.200,0.0,0.0,0.0
624,0.0,0.0,1.0,3.677,0.0,0.0,1.0
625,0.0,0.0,1.0,3.500,0.0,0.0,0.0


In [16]:
# Realizamos la descripción de la información previamente definida
table1.describe()

,american,instate,freshman,ACumGPA,greek,econ_hs,varsity
count,627.000000,627.000000,627.000000,627.000000,501.000000,501.000000,500.000000
mean,0.902711,0.221691,0.864434,3.432091,0.642715,0.578842,0.072000
std,0.296587,0.415717,0.342602,0.435175,0.479679,0.494239,0.258747
min,0.000000,0.000000,0.000000,1.200000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,1.000000,3.229000,0.000000,0.000000,0.000000
50%,1.000000,0.000000,1.000000,3.500000,1.000000,1.000000,0.000000
75%,1.000000,0.000000,1.000000,3.740000,1.000000,1.000000,0.000000
max,1.000000,1.000000,1.000000,4.000000,1.000000,1.000000,1.000000


In [18]:
# Realizamos una tabla resumen que contenga información sobre:  (mean, sd, min, max, count) 
# posteriormene transponemos la tabla
summary_table = table1.describe().loc[["mean","std","count","min","max"]].T
summary_table

,mean,std,count,min,max
american,0.902711,0.296587,627.0,0.0,1.0
instate,0.221691,0.415717,627.0,0.0,1.0
freshman,0.864434,0.342602,627.0,0.0,1.0
ACumGPA,3.432091,0.435175,627.0,1.2,4.0
greek,0.642715,0.479679,501.0,0.0,1.0
econ_hs,0.578842,0.494239,501.0,0.0,1.0
varsity,0.072000,0.258747,500.0,0.0,1.0


In [19]:
# Consideramos el nombre de cada colmuna para su posterior modificación
table1.columns

Index(['american', 'instate', 'freshman', 'ACumGPA', 'greek', 'econ_hs',
       'varsity'],
      dtype='object')

In [20]:
# Cambiando los nombres
table1.columns

new_names = ["Estudiante americano",
             "Estudiante in-state",
             "Estudiante es de primer año",
             "GPA acumulado",
             "Estudiante pertenece a una fraternidad o sororidad",
             "Estudiante llevó Economía en High School",
             "Estudiante es un atleta"]
# Realizamos la unión de la estrcutra bajo un diccionario
dict( zip( table1.columns, new_names) )

{'american': 'Estudiante americano',
 'instate': 'Estudiante in-state',
 'freshman': 'Estudiante es de primer año',
 'ACumGPA': 'GPA acumulado',
 'greek': 'Estudiante pertenece a una fraternidad o sororidad',
 'econ_hs': 'Estudiante llevó Economía en High School',
 'varsity': 'Estudiante es un atleta'}

In [22]:
# Optimizando la tabla de estadísticas descriptivas
nuevos_nombres = dict( zip( table1.columns, new_names) )

columns_nuevos_nombres = {
    "mean": "Mean",
    "std": "Standard Deviation",
    "count": "Observations",
    "min": "Min",
    "max": "Max"}

# Ahora la tabla tiene nuevos nombres tando de columnas como de filas
summary_table.rename(index=nuevos_nombres, columns=columns_nuevos_nombres, inplace=True)
summary_table

,Mean,Standard Deviation,Observations,Min,Max
Estudiante americano,0.902711,0.296587,627.0,0.0,1.0
Estudiante in-state,0.221691,0.415717,627.0,0.0,1.0
Estudiante es de primer año,0.864434,0.342602,627.0,0.0,1.0
GPA acumulado,3.432091,0.435175,627.0,1.2,4.0
Estudiante pertenece a una fraternidad o sororidad,0.642715,0.479679,501.0,0.0,1.0
Estudiante llevó Economía en High School,0.578842,0.494239,501.0,0.0,1.0
Estudiante es un atleta,0.072000,0.258747,500.0,0.0,1.0


In [24]:
# Exportamos a Latex
summary_table.style.format(subset="Mean", precision=2).format(subset="Standard Deviation", precision=2)\
.format(subset="Observations", precision=0).format(subset="Min", precision=0).format(subset="Max", precision=0)\
.to_latex(
    r"..\..\trabajo_final\Grupo11\Tabla_ED.tex",
caption="Descriptive Statistics",
    column_format = "lccc"   # l: left, c:center , r: right
)

### 2. Replica de tablas

#### 2.1 Tabla 3

In [74]:
# NO consideramos variables de control para el modelo 1
model1_3= "took_year ~ treat2016 + yr_2016 + treatment_class"

ols_model1 = smf.ols(model1_3, data=datos).fit(cov_type='cluster', cov_kwds={'groups': datos['took_year']})

print(ols_model1.summary())

                            OLS Regression Results                            
Dep. Variable:              took_year   R-squared:                       0.008
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                 6.005e+09
Date:                Thu, 13 Jul 2023   Prob (F-statistic):           9.49e-06
Time:                        18:30:18   Log-Likelihood:                -211.88
No. Observations:                 627   AIC:                             431.8
Df Residuals:                     623   BIC:                             449.5
Df Model:                           3                                         
Covariance Type:              cluster                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           0.1474      0.252     

In [68]:
# Considerando variables de control
control_vars = ["female_prof", "instate", "freshman", "american", "ACumGPA", "gradePrinciples","small_class"]
' + '.join( control_vars )

'female_prof + instate + freshman + american + ACumGPA + gradePrinciples + small_class'

In [75]:
# Tenemos la formula con las variables de control para el modelo 2
formula_model2_3 = "took_year ~ treat2016 + yr_2016 + treatment_class + " + ' + '.join( control_vars )
formula_model2_3

'took_year ~ treat2016 + yr_2016 + treatment_class + female_prof + instate + freshman + american + ACumGPA + gradePrinciples + small_class'

In [133]:
# Realizando la estimación
ols_model2 = smf.ols(formula_model2_3, data=datos).fit(cov_type='cluster', cov_kwds={'groups': datos['took_year']})
print(ols_model2.summary())

                            OLS Regression Results                            
Dep. Variable:              took_year   R-squared:                       0.035
Model:                            OLS   Adj. R-squared:                  0.020
Method:                 Least Squares   F-statistic:                -9.451e+09
Date:                Thu, 13 Jul 2023   Prob (F-statistic):               1.00
Time:                        19:54:11   Log-Likelihood:                -203.16
No. Observations:                 627   AIC:                             428.3
Df Residuals:                     616   BIC:                             477.2
Df Model:                          10                                         
Covariance Type:              cluster                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           0.3720      0.558     

In [134]:
# NO consideramos variables de control para el modelo 3
model3_3= "tookanother ~ treat2016 + yr_2016 + treatment_class"
ols_model3 = smf.ols(model3_3, data=datos).fit(cov_type='cluster', cov_kwds={'groups': datos['tookanother']})
print(ols_model3.summary())

                            OLS Regression Results                            
Dep. Variable:            tookanother   R-squared:                       0.010
Model:                            OLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                 4.359e+10
Date:                Thu, 13 Jul 2023   Prob (F-statistic):           3.52e-06
Time:                        19:54:21   Log-Likelihood:                -330.66
No. Observations:                 627   AIC:                             669.3
Df Residuals:                     623   BIC:                             687.1
Df Model:                           3                                         
Covariance Type:              cluster                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           0.2372      0.363     

In [82]:
# Tenemos la formula con las variables de control para el modelo 4
formula_model4_3 = "tookanother ~ treat2016 + yr_2016 + treatment_class + " + ' + '.join( control_vars )
formula_model4_3

'tookanother ~ treat2016 + yr_2016 + treatment_class + female_prof + instate + freshman + american + ACumGPA + gradePrinciples + small_class'

In [135]:
# Realizando la estimación
ols_model4= smf.ols(formula_model4_3, data=datos).fit(cov_type='cluster', cov_kwds={'groups': datos['tookanother']})
print(ols_model4.summary())

                            OLS Regression Results                            
Dep. Variable:            tookanother   R-squared:                       0.049
Model:                            OLS   Adj. R-squared:                  0.034
Method:                 Least Squares   F-statistic:                -2.136e+11
Date:                Thu, 13 Jul 2023   Prob (F-statistic):               1.00
Time:                        19:54:31   Log-Likelihood:                -318.16
No. Observations:                 627   AIC:                             658.3
Df Residuals:                     616   BIC:                             707.2
Df Model:                          10                                         
Covariance Type:              cluster                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           0.8032      0.927     

##### Proceso de exportación a latex

In [140]:
# Lista de explicativa a mostrarse en la tabla
explicativas = ['treat2016','yr_2016','treatment_class']

# etiquetas a las variables 
etiquetas = ['Treatment class x 2016','Year 2016','Treatment class (in 2015)']

labels = dict(zip(explicativas,etiquetas))
labels 

{'treat2016': 'Treatment class x 2016',
 'yr_2016': 'Year 2016',
 'treatment_class': 'Treatment class (in 2015)'}

In [144]:
pystout(models = [ols_model1,ols_model2,ols_model3,ols_model4], file=r"..\..\trabajo_final\Grupo11\Table_3x.tex", digits=3,
        endog_names=['Took Micro within year','Took another econclass','Took Micro within year','Took another econclass'],
        exogvars =explicativas ,  # sellecionamos las variables 
        varlabels = labels,  # etiquetas a las variables
        modstat={'nobs':'Observarions'}, # estadísticos 
        addrows={'Controls':['No','Yes','No','yes']},
        title='Treatment Effects on Intermediate Outcomes',
       )


#### 2.2 Tabla 4

In [142]:
# NO consideramos variables de control para el modelo 1
model1_4= "numeconclass ~ treat2016 + yr_2016 + treatment_class"

ols_model1 = smf.ols(model1_4, data=datos).fit(cov_type='cluster', cov_kwds={'groups': datos['class_fe2']})

print(ols_model1.summary())

                            OLS Regression Results                            
Dep. Variable:           numeconclass   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     3.964
Date:                Thu, 13 Jul 2023   Prob (F-statistic):             0.0385
Time:                        20:18:54   Log-Likelihood:                -1505.4
No. Observations:                 627   AIC:                             3019.
Df Residuals:                     623   BIC:                             3037.
Df Model:                           3                                         
Covariance Type:              cluster                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           1.0256      0.112     

In [138]:
# Tenemos la formula con las variables de control para el modelo 2
formula_model2_4 = "numeconclass ~ treat2016 + yr_2016 + treatment_class + " + ' + '.join( control_vars )
formula_model2_4

'numeconclass ~ treat2016 + yr_2016 + treatment_class + female_prof + instate + freshman + american + ACumGPA + gradePrinciples + small_class'

In [139]:
# Realizando la estimación
ols_model2 = smf.ols(formula_model2_4, data=datos).fit(cov_type='cluster', cov_kwds={'groups': datos['numeconclass']})
print(ols_model2.summary())

                            OLS Regression Results                            
Dep. Variable:           numeconclass   R-squared:                       0.064
Model:                            OLS   Adj. R-squared:                  0.049
Method:                 Least Squares   F-statistic:                     1.137
Date:                Thu, 13 Jul 2023   Prob (F-statistic):              0.406
Time:                        19:56:29   Log-Likelihood:                -1486.6
No. Observations:                 627   AIC:                             2995.
Df Residuals:                     616   BIC:                             3044.
Df Model:                          10                                         
Covariance Type:              cluster                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           5.8037      4.309     

In [91]:
# NO consideramos variables de control para el modelo 3
model3_4= "econmajor ~ treat2016 + yr_2016 + treatment_class"
ols_model3 = smf.ols(model3_4, data=datos).fit(cov_type='cluster', cov_kwds={'groups': datos['econmajor']})
print(ols_model3.summary())

                            OLS Regression Results                            
Dep. Variable:              econmajor   R-squared:                       0.009
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                 6.073e+09
Date:                Thu, 13 Jul 2023   Prob (F-statistic):           9.07e-06
Time:                        18:36:39   Log-Likelihood:                -133.19
No. Observations:                 627   AIC:                             274.4
Df Residuals:                     623   BIC:                             292.1
Df Model:                           3                                         
Covariance Type:              cluster                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           0.1026      0.185     

In [92]:
# Tenemos la formula con las variables de control para el modelo 4
formula_model4_4 = "econmajor ~ treat2016 + yr_2016 + treatment_class + " + ' + '.join( control_vars )
formula_model4_4

'econmajor ~ treat2016 + yr_2016 + treatment_class + female_prof + instate + freshman + american + ACumGPA + gradePrinciples + small_class'

In [94]:
# Realizando la estimación
ols_model4 = smf.ols(formula_model4_4, data=datos).fit(cov_type='cluster', cov_kwds={'groups': datos['tookanother']})
print(ols_model4.summary())

                            OLS Regression Results                            
Dep. Variable:              econmajor   R-squared:                       0.055
Model:                            OLS   Adj. R-squared:                  0.040
Method:                 Least Squares   F-statistic:                -1.803e+11
Date:                Thu, 13 Jul 2023   Prob (F-statistic):               1.00
Time:                        18:38:14   Log-Likelihood:                -118.28
No. Observations:                 627   AIC:                             258.6
Df Residuals:                     616   BIC:                             307.4
Df Model:                          10                                         
Covariance Type:              cluster                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           0.5105      0.635     

##### Proceso de exportación a latex

In [ ]:
# Lista de explicativa a mostrarse en la tabla
explicativas_4 = ['took_year','tookanother']
# etiquetas a las variables 
etiquetas_3 = ['Took Micro within year','Took another econclass']
labels = dict(zip(explicativas_3,etiquetas_3))
labels 

#### 2.3 Tabla 5

In [95]:
# Tenemos la formula con las variables de control para el modelo 1
formula_model1_5 = "Major_STEM ~ treat2016 + yr_2016 + treatment_class + " + ' + '.join( control_vars )
formula_model1_5

'Major_STEM ~ treat2016 + yr_2016 + treatment_class + female_prof + instate + freshman + american + ACumGPA + gradePrinciples + small_class'

In [97]:
# Realizando la estimación
ols_model1 = smf.ols(formula_model1_5, data=datos).fit(cov_type='cluster', cov_kwds={'groups': datos['Major_STEM']})
print(ols_model1.summary())

                            OLS Regression Results                            
Dep. Variable:             Major_STEM   R-squared:                       0.030
Model:                            OLS   Adj. R-squared:                  0.014
Method:                 Least Squares   F-statistic:                 3.612e+09
Date:                Thu, 13 Jul 2023   Prob (F-statistic):           1.28e-05
Time:                        18:41:50   Log-Likelihood:                -159.54
No. Observations:                 627   AIC:                             341.1
Df Residuals:                     616   BIC:                             389.9
Df Model:                          10                                         
Covariance Type:              cluster                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           0.2453      0.373     

In [98]:
# Tenemos la formula con las variables de control para el modelo 2
formula_model2_5 = "Major_Business ~ treat2016 + yr_2016 + treatment_class + " + ' + '.join( control_vars )
formula_model2_5

'Major_Business ~ treat2016 + yr_2016 + treatment_class + female_prof + instate + freshman + american + ACumGPA + gradePrinciples + small_class'

In [100]:
# Realizando la estimación
ols_model2 = smf.ols(formula_model2_5, data=datos).fit(cov_type='cluster', cov_kwds={'groups': datos['Major_Business']})
print(ols_model2.summary())

                            OLS Regression Results                            
Dep. Variable:         Major_Business   R-squared:                       0.022
Model:                            OLS   Adj. R-squared:                  0.007
Method:                 Least Squares   F-statistic:                 3.413e+09
Date:                Thu, 13 Jul 2023   Prob (F-statistic):           1.33e-05
Time:                        18:44:28   Log-Likelihood:                -319.96
No. Observations:                 627   AIC:                             661.9
Df Residuals:                     616   BIC:                             710.8
Df Model:                          10                                         
Covariance Type:              cluster                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -0.1535      0.149     

In [101]:
# Tenemos la formula con las variables de control para el modelo 3
formula_model3_5 = "Major_Finance ~ treat2016 + yr_2016 + treatment_class + " + ' + '.join( control_vars )
formula_model3_5

'Major_Finance ~ treat2016 + yr_2016 + treatment_class + female_prof + instate + freshman + american + ACumGPA + gradePrinciples + small_class'

In [102]:
# Realizando la estimación
ols_model3 = smf.ols(formula_model3_5, data=datos).fit(cov_type='cluster', cov_kwds={'groups': datos['Major_Finance']})
print(ols_model3.summary())

                            OLS Regression Results                            
Dep. Variable:          Major_Finance   R-squared:                       0.052
Model:                            OLS   Adj. R-squared:                  0.036
Method:                 Least Squares   F-statistic:                -5.780e+10
Date:                Thu, 13 Jul 2023   Prob (F-statistic):               1.00
Time:                        18:44:43   Log-Likelihood:                -224.50
No. Observations:                 627   AIC:                             471.0
Df Residuals:                     616   BIC:                             519.8
Df Model:                          10                                         
Covariance Type:              cluster                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -0.3188      0.466     

In [103]:
# Tenemos la formula con las variables de control para el modelo 4
formula_model4_5 = "Major_Marketing ~ treat2016 + yr_2016 + treatment_class + " + ' + '.join( control_vars )
formula_model4_5

'Major_Marketing ~ treat2016 + yr_2016 + treatment_class + female_prof + instate + freshman + american + ACumGPA + gradePrinciples + small_class'

In [104]:
# Realizando la estimación
ols_model4 = smf.ols(formula_model4_5, data=datos).fit(cov_type='cluster', cov_kwds={'groups': datos['Major_Marketing']})
print(ols_model4.summary())

                            OLS Regression Results                            
Dep. Variable:        Major_Marketing   R-squared:                       0.037
Model:                            OLS   Adj. R-squared:                  0.022
Method:                 Least Squares   F-statistic:                -8.163e+08
Date:                Thu, 13 Jul 2023   Prob (F-statistic):               1.00
Time:                        18:44:59   Log-Likelihood:                -199.30
No. Observations:                 627   AIC:                             420.6
Df Residuals:                     616   BIC:                             469.5
Df Model:                          10                                         
Covariance Type:              cluster                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -0.2548      0.380     

##### Proceso de exportación a latex

#### 2.4 Tabla 6

In [105]:
# Tenemos la formula con las variables de control para el modelo 1
formula_model1_6 = "Major_SocSc ~ treat2016 + yr_2016 + treatment_class + " + ' + '.join( control_vars )
formula_model1_6

'Major_SocSc ~ treat2016 + yr_2016 + treatment_class + female_prof + instate + freshman + american + ACumGPA + gradePrinciples + small_class'

In [106]:
# Realizando la estimación
ols_model1 = smf.ols(formula_model1_6, data=datos).fit(cov_type='cluster', cov_kwds={'groups': datos['Major_SocSc']})
print(ols_model1.summary())

                            OLS Regression Results                            
Dep. Variable:            Major_SocSc   R-squared:                       0.078
Model:                            OLS   Adj. R-squared:                  0.063
Method:                 Least Squares   F-statistic:                -7.000e+10
Date:                Thu, 13 Jul 2023   Prob (F-statistic):               1.00
Time:                        18:49:01   Log-Likelihood:                -131.82
No. Observations:                 627   AIC:                             285.6
Df Residuals:                     616   BIC:                             334.5
Df Model:                          10                                         
Covariance Type:              cluster                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           0.6238      0.883     

In [107]:
# Tenemos la formula con las variables de control para el modelo 2
formula_model2_6 = "Major_Arts ~ treat2016 + yr_2016 + treatment_class + " + ' + '.join( control_vars )
formula_model2_6

'Major_Arts ~ treat2016 + yr_2016 + treatment_class + female_prof + instate + freshman + american + ACumGPA + gradePrinciples + small_class'

In [108]:
# Realizando la estimación
ols_model2 = smf.ols(formula_model2_6, data=datos).fit(cov_type='cluster', cov_kwds={'groups': datos['Major_Arts']})
print(ols_model2.summary())

                            OLS Regression Results                            
Dep. Variable:             Major_Arts   R-squared:                       0.022
Model:                            OLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                -7.918e+09
Date:                Thu, 13 Jul 2023   Prob (F-statistic):               1.00
Time:                        18:50:52   Log-Likelihood:                 48.480
No. Observations:                 627   AIC:                            -74.96
Df Residuals:                     616   BIC:                            -26.11
Df Model:                          10                                         
Covariance Type:              cluster                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           0.0503      0.096     

In [109]:
# Tenemos la formula con las variables de control para el modelo 3
formula_model3_6 = "Major_Comm ~ treat2016 + yr_2016 + treatment_class + " + ' + '.join( control_vars )
formula_model3_6

'Major_Comm ~ treat2016 + yr_2016 + treatment_class + female_prof + instate + freshman + american + ACumGPA + gradePrinciples + small_class'

In [110]:
# Realizando la estimación
ols_model3 = smf.ols(formula_model3_6, data=datos).fit(cov_type='cluster', cov_kwds={'groups': datos['Major_Comm']})
print(ols_model3.summary())

                            OLS Regression Results                            
Dep. Variable:             Major_Comm   R-squared:                       0.021
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                 1.240e+09
Date:                Thu, 13 Jul 2023   Prob (F-statistic):           2.17e-05
Time:                        18:51:30   Log-Likelihood:                 31.395
No. Observations:                 627   AIC:                            -40.79
Df Residuals:                     616   BIC:                             8.060
Df Model:                          10                                         
Covariance Type:              cluster                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           0.1463      0.262     

In [111]:
# Tenemos la formula con las variables de control para el modelo 4
formula_model4_6 = "Major_Hum ~ treat2016 + yr_2016 + treatment_class + " + ' + '.join( control_vars )
formula_model4_6

'Major_Hum ~ treat2016 + yr_2016 + treatment_class + female_prof + instate + freshman + american + ACumGPA + gradePrinciples + small_class'

In [112]:
# Realizando la estimación
ols_model4 = smf.ols(formula_model4_6, data=datos).fit(cov_type='cluster', cov_kwds={'groups': datos['Major_Hum']})
print(ols_model4.summary())

                            OLS Regression Results                            
Dep. Variable:              Major_Hum   R-squared:                       0.038
Model:                            OLS   Adj. R-squared:                  0.023
Method:                 Least Squares   F-statistic:                -6.168e+08
Date:                Thu, 13 Jul 2023   Prob (F-statistic):               1.00
Time:                        18:52:02   Log-Likelihood:                -69.809
No. Observations:                 627   AIC:                             161.6
Df Residuals:                     616   BIC:                             210.5
Df Model:                          10                                         
Covariance Type:              cluster                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           0.1404      0.230     

##### Proceso de exportación a latex

###  3. Creación de un coefplot de la tabla 4